
/* QUESTIONS
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */


In [ ]:

SELECT name
FROM `Facilities`
WHERE membercost >0
#


/* Q2: How many facilities do not charge a fee to members? */


In [ ]:

SELECT COUNT(name)
FROM `Facilities`
WHERE membercost = 0
#


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */


In [ ]:

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > (monthlymaintenance*0.20)
#


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */


In [ ]:


CASE facid
WHEN 1 THEN 1
WHEN 2 THEN 2
END
FROM Facilities

#


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */



In [ ]:

SELECT name, monthlymaintenance,
CASE
	WHEN monthlymaintenance < 100 THEN 'cheap'
	WHEN monthlymaintenance > 100 THEN 'expensive'
	END AS CostType
FROM Facilities

#


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */



In [ ]:

SELECT firstname, surname
FROM `Members`
WHERE YEAR(joindate) >= 2012 AND MONTH(joindate) >= 9 AND DAY(joindate) >= 25;

#


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */



In [ ]:

SELECT DISTINCT CONCAT(m.firstname,' ', m.surname,' ', f.name )
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY m.surname

#


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */



In [ ]:

SELECT f.name AS Facility, CONCAT(m.firstname, ' ', m.surname) AS Name, CONCAT(f.membercost,'/', f.guestcost) AS cost
	FROM Bookings AS b
	INNER JOIN Facilities AS f
	ON b.facid= f.facid
	INNER JOIN Members AS m
     ON b.memid = m.memid WHERE b.starttime LIKE '2012-09-14%' AND (f.membercost > 30 OR f.guestcost > 30)
GROUP BY Cost DESC


#


/* Q9: This time, produce the same result as in Q8, but using a subquery. */



In [ ]:

SELECT * FROM
(SELECT f.name AS Facility, CONCAT(m.firstname, ' ', m.surname) AS Name, f.membercost AS Cost
	FROM Bookings AS b
	INNER JOIN Facilities AS f
	ON b.facid = f.facid
	INNER JOIN Members AS m
     ON b.memid = m.memid WHERE b.starttime LIKE '2012-09-14%'
UNION
SELECT f.name AS Facility, CONCAT(m.firstname, ' ', m.surname) AS Name, f.guestcost AS Cost
	FROM Bookings AS b
	INNER JOIN Facilities AS f
	ON b.facid = f.facid
	INNER JOIN Members AS m
     ON b.memid = m.memid
             WHERE b.starttime LIKE '2012-09-14%') AS df
WHERE df.Cost > 30
GROUP BY df.Cost DESC;
#

____________________________________________________________________
/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook
for the following questions.


In [2]:
import pandas as pd
import sqlite3
path = 'data/'
file = 'cc.db'
def query(query, file, path=''):
	"""Simple sqlite query function that returns the SQL query as a Pandas DataFrame"""
	with sqlite3.connect(path+file) as connection:
		cursor = connection.cursor()
		query = cursor.execute(query)
		return pd.DataFrame(query)
#

____________________________________________________________________
QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */


In [3]:

df = query(("""
SELECT facility, SUM(cost) FROM (
	SELECT f.name AS facility, SUM(f.membercost) AS cost
	FROM Facilities AS f
	INNER JOIN Bookings AS b
	ON b.facid = f.facid
    WHERE b.memid != 0
	GROUP BY facility
UNION ALL
	SELECT f.name AS facility, SUM(f.guestcost) AS cost
	FROM Facilities AS f
	INNER JOIN Bookings AS b
	ON b.facid = f.facid
    WHERE b.memid = 0
	GROUP BY facility) as db
WHERE cost < 1000
GROUP BY facility
ORDER BY cost DESC;
"""),file, path=path)
df.columns = ['Facility', 'Revenue']
df

#

,Facility,Revenue
0,Squash Court,682.5
1,Massage Room 2,267.3
2,Table Tennis,90.0
3,Snooker Table,115.0
4,Pool Table,265.0
5,Badminton Court,604.5


____________________________________________________________________
/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


In [4]:

df = query("""
SELECT m.surname, m.firstname, (r.surname || ' ' || r.firstname)
FROM Members AS m
LEFT JOIN Members AS r
ON r.recommendedby = m.memid
ORDER BY m.surname, m.firstname;
""", file, path=path)
df.columns = ['LastName', 'FirstName', 'RecommendedBy']
df
#

,LastName,FirstName,RecommendedBy
0,Bader,Florence,Sarwin Ramnaresh
1,Baker,Anne,None
2,Baker,Timothy,Coplin Joan
3,Boothe,Tim,None
4,Butters,Gerald,Genting Matthew
5,Coplin,Joan,None
6,Crumpet,Erica,None
7,Dare,Nancy,None
8,Farrell,David,None
9,Farrell,Jemima,Baker Timothy


____________________________________________________________________
/* Q12: Find the facilities with their usage by member, but not guests */


In [5]:

df = query("""
SELECT f.name AS facility, sub.usage AS usage
FROM Facilities AS f
JOIN (SELECT facid, COUNT(memid) AS usage
FROM Bookings
GROUP BY facid) AS sub
ON f.facid = sub.facid
ORDER by usage DESC
;""", file, path=path)
df.columns = ['Facility', 'UsageByMember']
df
#

,Facility,UsageByMember
0,Pool Table,836
1,Massage Room 1,629
2,Snooker Table,444
3,Squash Court,440
4,Tennis Court 1,408
5,Table Tennis,403
6,Tennis Court 2,389
7,Badminton Court,383
8,Massage Room 2,111


____________________________________________________________________
/* Q13: Find the facilities usage by month, but not guests */


In [6]:

df = query("""
SELECT f.name AS facility, sub.usage AS usage
FROM Facilities AS f
JOIN (SELECT facid, STRFTIME('%m', starttime) AS usage
FROM Bookings
WHERE memid != 0) AS sub
ON f.facid = sub.facid
ORDER by usage DESC
;""", file, path=path)
df.columns = ['Facility', 'UsageByMonth']

facs = list(df.set_index('Facility').index.unique())
d = {}
for fac in facs:
	d[fac] = df[df['Facility'] == fac].count()
df = pd.DataFrame(d)
df = df.transpose()
del df['Facility']
df

,UsageByMonth
Tennis Court 1,308
Tennis Court 2,276
Badminton Court,344
Table Tennis,385
Massage Room 1,421
Massage Room 2,27
Squash Court,195
Snooker Table,421
Pool Table,783
